In [1]:
import yfinance as yf
import pandas as pd
import requests
from bs4 import BeautifulSoup
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [2]:
import warnings
# Ignore all warnings
warnings.filterwarnings("ignore", category=FutureWarning)

In [3]:
def make_graph(stock_data, revenue_data, stock):
    fig = make_subplots(rows=2, cols=1, shared_xaxes=True, subplot_titles=("Historical Share Price", "Historical Revenue"), vertical_spacing = .3)
    stock_data_specific = stock_data[stock_data.Date <= '2021--06-14']
    revenue_data_specific = revenue_data[revenue_data.Date <= '2021-04-30']
    fig.add_trace(go.Scatter(x=pd.to_datetime(stock_data_specific.Date, infer_datetime_format=True), y=stock_data_specific.Close.astype("float"), name="Share Price"), row=1, col=1)
    fig.add_trace(go.Scatter(x=pd.to_datetime(revenue_data_specific.Date, infer_datetime_format=True), y=revenue_data_specific.Revenue.astype("float"), name="Revenue"), row=2, col=1)
    fig.update_xaxes(title_text="Date", row=1, col=1)
    fig.update_xaxes(title_text="Date", row=2, col=1)
    fig.update_yaxes(title_text="Price ($US)", row=1, col=1)
    fig.update_yaxes(title_text="Revenue ($US Millions)", row=2, col=1)
    fig.update_layout(showlegend=False,
    height=900,
    title=stock,
    xaxis_rangeslider_visible=True)
    fig.show()

In [4]:
def extract_data(ticker_name):
    company = yf.Ticker(ticker_name)
    company_data = company.history(period = "max")
    company_data.reset_index(inplace = True)
    company_data.head()
    return company_data

In [5]:
def webscraping(url):
    # html_data = requests.get(url)
    with open(url, 'r') as file:
        html_data = file.read()
    company_html = BeautifulSoup(html_data , 'html5lib') # If the code is saved locally remove .text
    data = []
    tables = company_html.find_all('table')
    for table in tables:
        if('Quarterly Revenue' in table.text):
            rows = table.find_all('tr')
            for row in rows:
                col = row.find_all('td')
                if col != []:
                    date = col[0].text
                    revenue = col[1].text.replace(',','').replace('$','')
                    data.append({"Date":date, "Revenue":revenue})
    company_revenue = pd.DataFrame(data, columns = ["Date","Revenue"])
    company_revenue.dropna(inplace=True)
    company_revenue = company_revenue[company_revenue['Revenue'] != ""]
    return company_revenue

In [6]:
import plotly.io as pio
pio.renderers.default = "browser"
ticker = "TSLA" 
data = extract_data(ticker)
# url = "https://www.macrotrends.net/stocks/charts/TSLA/tesla/revenue"
url = r"<address of the downloaded html>" # Using the address of saved html file 
revenue = webscraping(url)

In [ ]:
data.head(5)

In [ ]:
revenue.head(5)

In [ ]:
make_graph(data , revenue, "Graph for the company")